In [2]:
import torch

In [5]:
tt = torch.zeros(1200,64)
for i in range(64):
    tt[:, i] = i
tt[:3,:10]

tensor([[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]])

In [11]:
ttdim1 = torch.cumsum(tt, dim=1)
print(ttdim1.size())
print(ttdim1[:3,:10])

torch.Size([1200, 64])
tensor([[ 0.,  1.,  3.,  6., 10., 15., 21., 28., 36., 45.],
        [ 0.,  1.,  3.,  6., 10., 15., 21., 28., 36., 45.],
        [ 0.,  1.,  3.,  6., 10., 15., 21., 28., 36., 45.]])


In [12]:
ttdim0 = torch.cumsum(tt, dim=0)
print(ttdim0.size())
print(ttdim0[:3,:10])

torch.Size([1200, 64])
tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
        [ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18.],
        [ 0.,  3.,  6.,  9., 12., 15., 18., 21., 24., 27.]])


In [13]:
import numpy as np

In [14]:
tnp = tt.numpy()
tnp[:3,:10]

array([[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
       [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
       [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]], dtype=float32)

In [21]:
tnpax0 = np.cumsum(tnp, axis=0)
print(tnpax0.shape)
print(tnpax0[:3,:10])

(1200, 64)
[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
 [ 0.  2.  4.  6.  8. 10. 12. 14. 16. 18.]
 [ 0.  3.  6.  9. 12. 15. 18. 21. 24. 27.]]


In [22]:
tnpax1 = np.cumsum(tnp, axis=0)
print(tnpax1.shape)
print(tnpax1[:3,:10])

(1200, 64)
[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
 [ 0.  2.  4.  6.  8. 10. 12. 14. 16. 18.]
 [ 0.  3.  6.  9. 12. 15. 18. 21. 24. 27.]]
